
# <center>Ansatz Circuits for Approximate Quantum Compiling</center>

<center>Albert Akhriev and Anton Dekusar <br> IBM Research Ireland, 2023 </center>

## Introduction

Approximate Quantum Compiling (``AQC``) is a technique in quantum circuit design that aims to create approximate quantum circuits for a given quantum algorithm using a small set of gates. This approach is important because it can significantly reduce the computational resources needed to execute a quantum algorithm on a physical quantum computer, which is crucial given the limited number of qubits and coherence times currently available. In the context of framework presented in this package, we consider two particular flavours of AQC approach:
1. Given a $2^n{\times}2^n$ unitary matrix $U$, find $n$-qubit quantum circuit that approximates the target $U$. This is a generic technique, which does not impose any restrictions on target matrix, although the approximation accuracy of resultant quantum circuit and its depth greatly depend on $U$, see [1, 2] for more precise formulation.
2. Given initial state, specific Hamiltonian ($XXX$ spin-chain) and its corresponding Trotterized quantum circuit, find an approximate circuit having a smaller depth but still be capable to evolve the initial state forward in time with reasonable accuracy. The idea is to make the best use of the present days, noisy quantum computers, see [[4],[5]](#references) for details. We also refer this formulation as Approximate State Preparation (``ASP``).

In both cases, the problem is solved by optimizing a parametrized ansatz circuit. For better understanding of optimization process, this notebook demonstrates the basic principles of circuit design adopted in this framework. In the next section, we shall briefly outline the generic AQC approach, and in the subsequent section present the layout of approximating ansatz. In the last section, the ``Trotterized`` ansatz for ASP and time-evolution will be described.

#### Module paths

If running this notebook without installation, directly *from the root folder* of the package downloaded from the GitHub, the following ``bash`` snippet helps to resolve module paths:

```
(export PYTHONPATH=`pwd` && echo "Python path: ${PYTHONPATH}" && jupyter notebook docs/problem_and_ansatz.ipynb)
```

## Optimization Problem

In this framework, we focus on target unitary matrices to be compiled in the space of special unitary matrices of dimension $d$, $SU(d)$, where $d = 2^n$ and $n$ is the number of qubits. In particular, we considered random unitary target circuits which require a circuit length of $L \ge \frac{1}{4}(4^n - 3n - 1)$ in order to be exactly compiled, [[1],[2]](#references).
We are interested in compiling a quantum circuit, which we formalize as finding the "best" circuit representation in terms of an ordered gate sequence of a target unitary matrix $U \in U(d)$, with some additional hardware constraints. In particular, we look at representations that could be constrained in terms of hardware connectivity, as well as circuit length, and we choose a gate basis in terms of CNOT and rotation gates.

Here we follow on the recent papers [[1],[3]](#references), which formulate the problem as a mathematical optimization program over properly parameterized hardware-compatible circuits. In particular, in [[1]](#references), one defines a target circuit as a unitary matrix in $n$ qubits, $U$, and a parametric ansatz $V(\Theta)$ built upon allowed gates and interconnections, and solves (classically) the optimization problem. 

More precisely: Given a target *special unitary* matrix $U \in SU(2^n)$ and a set of constraints, in terms of connectivity and length, find the closest special unitary matrix $V \in {\cal V} \subseteq SU(2^n)$, where ${\cal V}$ represents the set of special unitary matrices that can be realized with rotations and CNOT gates alone and satisfy both connectivity and length constraints, by solving the following mathematical program: 
$$
\Theta^* = \arg \min_{\Theta} f(\Theta) := \left\|V(\Theta) - U\right\|^2_F , 
\qquad V(\Theta) \in {\cal V} \subseteq SU(2^n)\qquad\qquad\qquad (1)
$$
where $\|.\|_F$ denotes the Frobenious norm and $d = 2^n$. We call (1) the approximate quantum compiling (master) problem (AQCP). A solution of the problem is an optimal $V(\Theta^*)$ indicated as $V^*$, along with an ordered set of gate operations that respect the constraints.

The alternative formulation, aiming to reduce computational complexity, has been presented in [[2]](#references), which we call a *sketching* approach. Let a complex matrix $Q \in C^{d{\times}m}$, $m \le d$, have orthonormal columns. Then, we have that
$$
f^{(sk)}(\Theta) := \frac{1}{2m}\left\|(V - U)QQ^{\dagger}\right\|^2_F =
1 - \frac{1}{m}\text{Re} \left\langle V Q, UQ \right\rangle \in [0,2]. \qquad\qquad\qquad (2)
$$
Note that for the special case $m = d$ and $Q = I$, we get exactly the Problem (1) in equivalent formulation:
$$
f^{(full)}(\Theta) := 1 - \frac{1}{d}\text{Re} \left\langle V, U \right\rangle \in [0,2]. \qquad\qquad\qquad (3)
$$
In case of general unitary matrix $U$ we transform it into a special unitary $(SU(d))$ one before optimization: $\widetilde{U} := U / det(U)^{\frac{1}{d}}$ and $det(\widetilde{U}) = 1$, and use $\widetilde{U}$ as a target in (1) - (3). Then, the phase $\phi$ between approximation $V(\Theta)$ and $U$ is computed after optimization, such that $e^{i \phi} V(\Theta^*) \approx U$, and included in the resultant quantum circuit (instance of ```QuantumCircuit``` class) as a global phase.



## Parametrized Ansatz Circuit

We follow [[3]](#references) in restricting the parametric ansatz to have a special spin structure with a number of parameters that grows linearly in the number of qubits $n$. The matrix of parametric ansatz is denoted as $V = V(\Theta)$, where $\Theta$ is a vector of angular parameters. In [[2]](#references) we made yet another assumption: the target matrices $U$ are generated by taking random values of $θ$ in some parametric ansatz we choose, i.e., matrices that have the *same* structure as the matrix $V(\Theta)$ (so we know that we can exactly compile them, if we are able to solve Problem (1) to optimality).

Utilities for handling parametrized ansatz are grouped in 3 modules:
* ```aqc_research.parametric_circuit.py``` defines a class ```ParametricCircuit``` with many useful member functions.
* ```aqc_research.circuit_structures.py``` hosts utilies for circuit creation and related ones.
* ```aqc_research.circuit_transform.py``` is about transforming a circuit into different representations.

The following elementary structure is called a *unit-block*. Three types of unit-block are currently supported. They differ by entangling gate: CNOT, CZ or CPhase. The first two unit-blocks have 4 angular parameters (one per 1-qubit rotation gate) and the third one has additional phase parameter of CPhase gate (5 parameters in total):


In [ ]:
import sys
import logging 
import numpy as np
from matplotlib import pyplot as plt
from qiskit import QuantumCircuit
from aqc_research.parametric_circuit import ParametricCircuit
from aqc_research.circuit_structures import create_ansatz_structure, num_blocks_per_layer
from aqc_research.circuit_transform import ansatz_to_qcircuit
logging.basicConfig(stream=sys.stdout, level=logging.INFO, format="%(asctime)s %(message)s", datefmt="%Y-%m-%y %H:%M:%S")
np.random.seed(1234567)

print("Unit-blocks of parametrized ansatz with different entangling gates:")
fig, axs = plt.subplots(ncols=3, nrows=1, sharey=True, figsize=(15,15), gridspec_kw={"width_ratios": [1, 1, 1.25]})
fig.tight_layout()
for i, ent in enumerate(["cx", "cz", "cp"]):
    axs[i].title.set_text(("CNOT" if i == 0 else ("CZ" if i == 1 else "CPhase")) + " entangler")
    one_block = create_ansatz_structure(num_qubits=2, depth=1)
    circ = ParametricCircuit(num_qubits=2, entangler=ent, blocks=one_block)
    thetas = np.round(2 * np.pi * (0.01 + 0.98 * np.random.rand(circ.num_thetas)), 2)
    thetas[0 : 6] = 0  # drops front layer
    qc = ansatz_to_qcircuit(circ=circ, thetas=thetas, tol=1e-8)
    qc.draw(output="mpl", ax=axs[i])

Let us now have a look at the full ansatz circuit. It always starts from triplets of 1-qubit gates applied to each qubit, as justified in [[1],[2]](#references). We call the collection of 1-qubit triplets the ```front layer```. 

The notion of ```connectivity``` pertains to the physical affinity of qubits on real hardware. Here we support two types of connectivity: "full", when a 2-qubit gate can connect arbitrary pair of qubits, and "line", when a gate act on adjacent qubits only. In the context of this framework both connectivity types are equivalent, that is, we always connect adjacent qubits by a 2-qubit unit-block. We admit one exception in case of "line" connectivity, namely, the first and the last qubits can still be connected for certain hardware configurations, but any other pair cannot.

There are 4 options for ansatz ```layout```: "spin", "line", "cyclic_spin" and "cyclic_line". We shall focus in this presentation on "spin" and "cyclic_spin" options. The "spin" layout resembles the chessboard. The first and the last qubits remain unconnected. The "cyclic_spin" layout looks similar but in addition the first and the last qubits are connected by a unit-block. See the examples below.

It is worth to mention few other structural notions. ```Depth``` is the total number of 2-qubit unit-block in a circuit. User should specify it explicitly in ```create_ansatz_structure()``` function. Alternatively, the function ```fraction_of_lower_bound()``` from the module ```circuit_structures``` returns the circuit depth as a fraction of the lower-bound length $L = \frac{1}{4}(4^n - 3n - 1)$, see [[1],[2]](#references), which should be used cautiously to avoid exponentially long circuits.

```Blocks``` is a $2{\times}L$ Numpy array of 2-qubit block placements, where $L$ is the circuit depth. The first and the second rows hold indices of control and target qubits respectively. In fact, ```blocks``` encodes the circuit structure in a concise way.

Experimenting with the examples below, one can notice that 2-qubit block structure repeats after ```num_qubits-1``` blocks for "spin", "line" layouts and after ```num_qubits``` blocks for "cyclic_spin" and "cyclic_line". It make sense to define the concept of ```layer``` as a repeating subset of unit-blocks (of size *num_qubits-1* or *num_qubits* respectively). The function ```num_blocks_per_layer()``` returns the number of unit-blocks in a layer depending on the number of qubits and circuit layout. 

The code below generates some circuit examples:

In [ ]:
def make_exemplar_ansatz(num_qubits: int, layout: str, num_layers: int) -> QuantumCircuit:
    blocks = create_ansatz_structure(
        num_qubits=num_qubits, layout=layout, connectivity="full",
        depth=num_layers * num_blocks_per_layer(num_qubits, layout),  # depth of 'num_layers' layers
        logger=logging.getLogger()
    )
    circ = ParametricCircuit(num_qubits, entangler="cx", blocks=blocks)
    # Note, if theta[i] == 0, the corresponding i-th gate will be discarded.
    thetas = np.round(2 * np.pi * (0.01 + 0.98 * np.random.rand(circ.num_thetas)), 2)
    qc = ansatz_to_qcircuit(circ=circ, thetas=thetas)
    return qc

make_exemplar_ansatz(num_qubits=5, layout="spin", num_layers=2).draw(output="mpl")
# draw(output="mpl", style={"dpi": 100}, fold=-1)

"Cyclic_spin" is also a layered layout, although spotting the layers might be not easy:

In [ ]:
make_exemplar_ansatz(num_qubits=5, layout="cyclic_spin", num_layers=2).draw(output="mpl")

## Trotterized Ansatz

The first order Trotter circuit, in the context of paper [[2]](#references), has the following structure.

In [ ]:
from aqc_research.model_sp_lhs.trotter.trotter import trotter_circuit

num_qubits = 5
qc = trotter_circuit(QuantumCircuit(num_qubits), dt=0.4, delta=1.0, num_trotter_steps=2, second_order=False)
qc.draw(output="mpl", scale=0.7, fold=-1)

In the second order Trotter circuit the leading and training *half-layers* have different angular parameters comparing to the internal layers.

In [ ]:
num_qubits = 5
qc = trotter_circuit(QuantumCircuit(num_qubits), dt=0.4, delta=1.0, num_trotter_steps=2, second_order=True)
qc.draw(output="mpl", scale=0.7, fold=-1)

Please, refer to the accompanying notebook ``time_evolution.ipynb`` for more detailed description of Trotterized circuit and ansatz. In a nutshell, Trotter circuit consists of 2-qubit blocks of the following structure combined into layers of chessboard-like pattern, where every two adjacent qubits are connected by a 2-qubit block.

In [ ]:
qc = trotter_circuit(qc=QuantumCircuit(2), dt=0.4, delta=1.0, num_trotter_steps=1, second_order=False)
qc.draw(output="mpl")

The *Trotterized ansatz*, we use to approximate the true Trotter circuit, has the similar structure. It can be obtained from generic parametric circuit, presented above, by placing every 3 unit-blocks in a row and swapping the control/target qubits in the middle block. The picture below demonstrates *extended unit-block* (triplet block) of Trotterized ansatz.

In [ ]:
from aqc_research.parametric_circuit import TrotterAnsatz
from aqc_research.circuit_structures import make_trotter_like_circuit

blocks = make_trotter_like_circuit(num_qubits=2, num_layers=1)
circ = TrotterAnsatz(num_qubits=2, blocks=blocks, second_order=False)
thetas = np.round(2 * np.pi * (0.01 + 0.98 * np.random.rand(circ.num_thetas)), 2)
thetas[0 : 6] = 0  # drops the front layer
qc = ansatz_to_qcircuit(circ=circ, thetas=thetas, tol=1e-8)
qc.draw(output="mpl")

Finally, let us show a complete Trotterized ansatz with the front layer of 1-qubit gates.

In [ ]:
num_qubits = 5
blocks = make_trotter_like_circuit(num_qubits=num_qubits, num_layers=2)
circ = TrotterAnsatz(num_qubits=num_qubits, blocks=blocks, second_order=False)
thetas = np.round(2 * np.pi * (0.01 + 0.98 * np.random.rand(circ.num_thetas)), 2)
qc = ansatz_to_qcircuit(circ=circ, thetas=thetas, tol=1e-8)
qc.draw(output="mpl")

# References

1. L. Madden, and A. Simonetto, "Best approximate quantum compiling problems," ACM Transactions on Quantum Computing, vol.3, no.2, pp.1-29, 2022, https://arxiv.org/pdf/2106.05649.pdf
2. L. Madden, A. Akhriev and A. Simonetto, "Sketching the Best Approximate Quantum Compiling Problem," 2022 IEEE International Conference on Quantum Computing and Engineering (QCE), https://arxiv.org/abs/2205.04025
3. S. Khatri, R. LaRose, A. Poremba, L. Cincio, A.T. Sornborger, and P.J. Coles, "Quantum-assisted quantum compiling," Quantum, vol. 3, p. 140, 2019.
4. N.F. Robertson, A. Akhriev, J. Vala, S. Zhuk, "Escaping barren plateaus in approximate quantum compiling," https://arxiv.org/abs/2210.09191
5. N.F. Robertson, A. Akhriev, J. Vala, S. Zhuk, "Approximate Quantum Compiling for Quantum Simulation: A Tensor Network based approach," https://arxiv.org/abs/2301.08609